In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint

In [32]:
import json
import urllib3

In [33]:
# Import API key
from api_keys import audio_api

In [46]:
artists = ["coldplay", "aerosmith", "neko%&case", "radiohead"]

In [47]:
# EXAMPLE theaudiodb.com/api/v1/json/1/search.php?s=coldplay

audio_data = {'artist':[],
              'style':[], 
              'genre':[]
             }

#loop over artists dictionary and add data to dictionary
for artist in artists:
    base_url = "http://theaudiodb.com/api/v1/json/"+str(audio_api)+"/search.php?s="

    try:
        req = requests.get(base_url + f'{artist}')
        data = req.json()
        #url = urllib3.urlopen(base_url + f'{artist}')
        #data = json.load(url)
        
        for item in data['artists']:
        #add values to dictionary
            #print(["strArtist"])
            #print(["strStyle"])
            #print(["strGenre"])
        
            audio_data['artist'].append(item["strArtist"])
            audio_data['style'].append(item["strStyle"])
            audio_data['genre'].append(item["strGenre"])
            
            #audio_data['artist'].append(item["artists"]["strArtist"])
            #audio_data['style'].append(item["artists"]["strStyle"])
            #audio_data['genre'].append(item["artists"]["strGenre"])
        
        #print(base_url + f'{artist}')
        
    except:
        pass

In [48]:
print(audio_data)

{'artist': ['Coldplay', 'Aerosmith', 'NekoJam', 'Neko Case', 'Neko Case & Her Boyfriends', 'Radiohead'], 'style': ['Rock/Pop', 'Rock/Pop', None, '', None, 'Rock/Pop'], 'genre': ['Alternative Rock', 'Hard Rock', '', '', '', 'Alternative Rock']}


{'artists': [{'idArtist': '111268', 'strArtist': 'Aerosmith', 'strArtistStripped': None, 'strArtistAlternate': '', 'strLabel': 'Columbia', 'idLabel': '43473', 'intFormedYear': '1970', 'intBornYear': '1970', 'intDiedYear': None, 'strDisbanded': None, 'strStyle': 'Rock/Pop', 'strGenre': 'Hard Rock', 'strMood': 'Philosophical', 'strWebsite': 'www.aerosmith.com', 'strFacebook': 'www.facebook.com/aerosmith', 'strTwitter': 'twitter.com/Aerosmith', 'strBiographyEN': 'Aerosmith is an American rock band, sometimes referred to as "The Bad Boys from Boston" and "America\'s Greatest Rock and Roll Band." Their style, which is rooted in blues-based hard rock, has come to also incorporate elements of pop, heavy metal, and rhythm and blues, and has inspired many subsequent rock artists. The band was formed in Boston, Massachusetts in 1970. Guitarist Joe Perry and bassist Tom Hamilton, originally in a band together called the Jam Band, met up with singer Steven Tyler, drummer Joey Kramer, and guitarist

In [44]:
#base_url = "http://theaudiodb.com/api/v1/json/"+str(audio_api)+"/search.php?s="
#url = urllib3.urlopen(base_url + f'{artist}')
#data = json.load(url)

#for item in data['artists']:
#    print(item['strArtist'])

In [45]:
print(base_url)

http://theaudiodb.com/api/v1/json/523532/search.php?s=


In [45]:
#print(audio_api)

In [ ]:
# https://www.youtube.com/watch?v=pxofwuWTs7c
# this turorital has been helpful